<font color=#5559AB> **1.1 Study Area** </font>

<font color=black> This section utilizes BasinMaker, developed by Han et al. (2021), to extract/format a shapefile of your study area to be used throughout the Magpie Workflow. The shapefile can be generated by either the gauge name or latitude and longitude. Don't have that information? No problem, the first section of the workbook helps to identify the coordinates and gauge name. </font>

Check out the short video [2.1 Study Area - Magpie Workflow](https://youtu.be/QvAY-ZTFGBs) for more information

In [1]:
from google.colab import drive
drive.mount('/content/google_drive',force_remount=True)

Mounted at /content/google_drive


In [2]:
!python -m pip install geopandas==0.13.0 &> /dev/null
!python -m pip install ipyleaflet==0.17.2 &> /dev/null
!python -m pip install wget==3.2 &> /dev/null
!python -m pip install rasterstats==0.19.0 &> /dev/null
!python -m pip install simpledbf==0.2.6 &> /dev/null
!python -m pip install rioxarray==0.14.1 &> /dev/null
!python -m pip install fiona==1.9.4 &> /dev/null
!python -m pip install --target=$nb_path basinmaker==3.0.3 &> /dev/null
#!python -m pip install https://github.com/dustming/basinmaker/archive/master.zip &> /dev/null

In [3]:
import pandas as pd
import numpy as np
import os
import json
import geopandas as gpd
import shutil
import matplotlib.pyplot as plt
from glob import glob
#import rioxarray as rxr
#import wget
import zipfile
import fiona
# getting coordinated of study area
from geopy.geocoders import Nominatim
# interactive map
import folium
import branca
# BasinMaker
from basinmaker import basinmaker
from basinmaker.postprocessing.plotleaflet import plot_routing_product_with_ipyleaflet
from basinmaker.postprocessing.downloadpd import Download_Routing_Product_For_One_Gauge
from basinmaker.postprocessing.downloadpdptspurepy import Download_Routing_Product_From_Points_Or_LatLon
import time
import rasterstats as rs

In [4]:
# function to use coordinates to download routing producting
# user can either define coords or use the city name and python library "geolocator" to determine coordinates
def studyArea_location(city_name, define_lat, define_lon):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(1) Collect study area location information')
    print('-----------------------------------------------------------------------------------------------')
    # assign lat and lon to either found or identified coordinates
    if city_name != 'NA' and define_lat == 'NA':
        # determine coordinates from city name using geopy.geocoders library
        geolocator = Nominatim(user_agent="Magpie")
        location = geolocator.geocode(city_name)
        found_lat = location.latitude
        found_lon = location.longitude
        lat,lon = found_lat, found_lon,
        print("Study area coordinates:", lat,lon)
        return(lat,lon)
    # use defined coordinates
    elif city_name == 'NA' and define_lat != 'NA':
        lat,lon = float(define_lat), float(define_lon),
        print("Study area coordinates:", lat,lon)
        return(lat,lon)
    # neither coords nor city name are defined, encourages user to define gauge name
    elif city_name and define_lat == "NA":
        lat = None
        print("Define gauge name")

In [5]:
# if "interactive_map_response": "yes"
# an interative map is generated to visualize which gauges are near the previously defined coords
def interactive_gauge_map(interactive_map_response,lat,lon):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(1a) Interactive gauge plot')
    print('-----------------------------------------------------------------------------------------------')
    print('Response: ', interactive_map_response)
    if interactive_map_response == 'yes':
        # read in csv with gauge information
        gauge_info = pd.read_csv(os.path.join(variable['main_dir'], 'extras', 'subbasin_plots', f'obs_gauges_NA_v2-1.csv'))

        # format fancy folium pop-up
        def fancy_html(row):
            i = row
            subId = gauge_info['SubId'].iloc[i]
            obs_gauge = gauge_info['Obs_NM'].iloc[i]
            lat_info = gauge_info['POINT_Y'].iloc[i]
            lon_info = gauge_info['POINT_X'].iloc[i]
            sub_Reg = gauge_info['Sub_Region'].iloc[i]
            html = """<!DOCTYPE html>
        <html>
        <p>SubID: {}</td>""".format(subId) + """</p>
        <p>Obs Gauge: {}</td>""".format(obs_gauge) + """</p>
        <p>Lat: {}</td>""".format(lat_info) + """</p>
        <p>Lon: {}</td>""".format(lon_info) + """</p>
        <p>Sub Region: {}</td>""".format(sub_Reg) + """</p>
        </html>
        """
            return html

        # generate map with rectangular guide to assist in identifying the ideal subbasin/gauges to use
        if lat is None:
          print("Define gauge name in cell above")
        else:
          grid_pt=(lat,lon)
          W=grid_pt[1]-0.5
          E=grid_pt[1]+0.5
          N=grid_pt[0]+0.5
          S=grid_pt[0]-0.5

          upper_left=(N,W)
          upper_right=(N,E)
          lower_right=(S,E)
          lower_left=(S,W)

          line_color='red'
          fill_color='red'
          weight=2
          text='text'
          edges = [upper_left, upper_right, lower_right, lower_left]

          map_osm = folium.Map(location=[lat, lon],zoom_start=9)
          folium.LatLngPopup().add_to(map_osm)

          for i in range(0,len(gauge_info)):
            html = fancy_html(i)
            iframe = branca.element.IFrame(html=html,width=200,height=200)
            popup = folium.Popup(iframe,parse_html=True)
            # adds markers for each gauge
            folium.Marker([gauge_info['POINT_Y'].iloc[i],gauge_info['POINT_X'].iloc[i]],popup=popup).add_to(map_osm)
        # displays interactive map
        display(map_osm.add_child(folium.vector_layers.Polygon(locations=edges, color=line_color, fill_color=fill_color,weight=weight, popup=(folium.Popup(text)))))
    elif interactive_map_response == 'no':
        print('\nIf you would like to visualize the interactive gauge map, change your response to in the configuration_file.json for \"interactive_map_response\" to \"yes\"')

In [6]:
# downloads BasinMaker routing product with lat and lon
def download_routing_product_lat_lon(product_name,lat,lon):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(2) Download routing product')
    print('-----------------------------------------------------------------------------------------------')
    print('product_name: ', product_name)

    coords = pd.DataFrame({'lat': [lat], 'lon': [lon]})
    subid,product_path = Download_Routing_Product_From_Points_Or_LatLon(product_name = product_name,Lat =coords['lat'],Lon = coords['lon'])
    print('Successfully downloaded routing product using lat and lon!')
    return(product_path)

In [7]:
# downloads BasinMaker routing product with gauge name
def download_routing_product_gauge(product_name,gauge_name):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(2) Download routing product')
    print('-----------------------------------------------------------------------------------------------')
    print('product_name: ', product_name)
    print('gauge_name: ', gauge_name)

    subid,product_path = Download_Routing_Product_For_One_Gauge(gauge_name = gauge_name,product_name = product_name)
    print('Successfully downloaded routing product using the gauge name!')
    return(product_path)

In [8]:
# extracts drainage area
def extract_drainage_area(most_down_stream_subbasin_ids, most_up_stream_subbasin_ids, product_path):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(3) Extract drainage area and simplify drainage product')
    print('-----------------------------------------------------------------------------------------------')
    if most_down_stream_subbasin_ids == 'NA':
      ID_response = input("Enter most down stream subbasin ID: ")
      most_down_stream_subbasin_ids = ID_response
    else:
      print('Most down stream subbasin ID is: ', most_down_stream_subbasin_ids)
    most_down_stream_subbasin_ids_lst = [most_down_stream_subbasin_ids]
    most_up_stream_subbasin_ids_lst = [most_up_stream_subbasin_ids]

    # define the folder path for downloaded and unziped lake river routing prodcut folder, where several GIS files exist
    unzip_routing_product_folder = product_path

    # define another folder that will save the outputs
    folder_product_for_interested_gauges=os.path.join(variable['temporary_dir'],f'catchment_extraction_{most_down_stream_subbasin_ids}')
    # if folder doesn's exist
    if not os.path.exists(folder_product_for_interested_gauges):
      os.makedirs(folder_product_for_interested_gauges)

    # Initialize the basinmaker
    start = time.time()
    bm = basinmaker.postprocess()

    # extract subregion of the routing product
    bm.Select_Subregion_Of_Routing_Structure(
        path_output_folder = folder_product_for_interested_gauges,
        routing_product_folder = unzip_routing_product_folder,
        most_down_stream_subbasin_ids=most_down_stream_subbasin_ids_lst,
        most_up_stream_subbasin_ids=most_up_stream_subbasin_ids_lst,               # -1: extract to the most-upstream (headwater) subbasin; other subbasin ID: extract the areas from the outlet to the provided subbasin.
        gis_platform="purepy",
    )
    end = time.time()
    print("This section took  ", end - start, " seconds")

    # version number define
    version_num = variable['version_num_shp']

    # read in study area
    studyArea_bound = gpd.read_file(os.path.join(folder_product_for_interested_gauges,f'catchment_without_merging_lakes_{version_num}.shp'))
    studyArea_bound["dissolve"] = 1

    boundary = studyArea_bound[['dissolve', 'geometry']]
    cont_studyArea = boundary.dissolve(by='dissolve')

    # generate drive folder
    shp_dir = os.path.join(variable['main_dir'], 'shapefile')
    shp_path = os.path.isdir(shp_dir)
    if not shp_path:
      os.makedirs(shp_dir)
      print("created  folder: ", shp_dir)

    # save to drive
    cont_studyArea.to_file(os.path.join(shp_dir,'studyArea_outline.shp'))

    print('\n-----------------------------------------------------------------------------------------------')
    print('(4) Shapefile complete!')
    print('-----------------------------------------------------------------------------------------------')

In [9]:
# extracts drainage area for CLRH product
def extract_drainage_area_CLRH(most_down_stream_subbasin_ids, most_up_stream_subbasin_ids, product_path):
    print('\n-----------------------------------------------------------------------------------------------')
    print('(3) Extract drainage area and simplify drainage product')
    print('-----------------------------------------------------------------------------------------------')
    if most_down_stream_subbasin_ids == 'NA':
      ID_response = input("Enter most down stream subbasin ID: ")
      most_down_stream_subbasin_ids = ID_response
    else:
      print('Most down stream subbasin ID is: ', most_down_stream_subbasin_ids)
    most_down_stream_subbasin_ids_lst = [most_down_stream_subbasin_ids]
    most_up_stream_subbasin_ids_lst = [most_up_stream_subbasin_ids]

    print(product_path)

    # the downloaded routing product will be stored under product_path
    # the purpose of following code is just to move the downloaded product to a folder
    watershed_name = variable['gauge_name']
    folder_product_for_interested_gauges_CLRH =os.path.join(os.getcwd(),watershed_name,'extract_CLRH')
    if not os.path.exists(folder_product_for_interested_gauges_CLRH):
      os.makedirs(folder_product_for_interested_gauges_CLRH)

    # find the folder that containing the shapefiles
    for root, dirs, files in os.walk(product_path):
        for file in files:
          if ".shp" in file:
            file_path = os.path.join(root, file)
            product_dir = os.path.dirname(file_path)
            break

    # Get a list of all items in the source folder
    items = os.listdir(product_dir)

    # Iterate over each item
    for item in items:
        # Get the full path of the item
        item_path = os.path.join(product_dir, item)
        # Move the item to the destination folder
        shutil.move(item_path, folder_product_for_interested_gauges_CLRH)

    # define the folder path for downloaded and unziped lake river routing prodcut folder, where several GIS files exist.
    unzip_routing_product_folder= product_path         # example  unzip_routing_product_folder= product_path, this variable is defined in the last section

    # define another folder that will save the outputs
    watershed_name = variable['gauge_name_shp']
    folder_product_for_interested_gauges=os.path.join(os.getcwd(),watershed_name,'extract_CLRH')

    # Initialize the basinmaker
    start = time.time()
    bm = basinmaker.postprocess()

    # extract subregion of the routing product
    bm.Select_Subregion_Of_Routing_Structure(
        path_output_folder = folder_product_for_interested_gauges,
        routing_product_folder = unzip_routing_product_folder,
        most_down_stream_subbasin_ids=most_down_stream_subbasin_ids_lst,
        most_up_stream_subbasin_ids=most_up_stream_subbasin_ids_lst,               # -1: extract to the most-upstream (headwater) subbasin; other subbasin ID: extract the areas from the outlet to the provided subbasin.
        gis_platform="purepy",
    )
    end = time.time()
    print("This section took  ", end - start, " seconds")

        # version number define
    version_num = variable['version_num']

    # read in study area
    studyArea_bound = gpd.read_file(os.path.join(folder_product_for_interested_gauges,f'catchment_without_merging_lakes_{version_num}.shp'))
    studyArea_bound["dissolve"] = 1

    boundary = studyArea_bound[['dissolve', 'geometry']]
    cont_studyArea = boundary.dissolve(by='dissolve')

    # generate drive folder
    shp_dir = os.path.join(variable['main_dir'], 'shapefile')
    shp_path = os.path.isdir(shp_dir)
    if not shp_path:
      os.makedirs(shp_dir)
      print("created  folder: ", shp_dir)

    # save to drive
    cont_studyArea.to_file(os.path.join(shp_dir,'studyArea_outline.shp'))

    print('\n-----------------------------------------------------------------------------------------------')
    print('(4) Shapefile complete!')
    print('-----------------------------------------------------------------------------------------------')

In [10]:
def intersecting_polygons(shp_file_path,product_path):
  print('\n-----------------------------------------------------------------------------------------------')
  print('(3a) Check projection')
  print('-----------------------------------------------------------------------------------------------')
  # find name of shapefile
  for shp_file in os.listdir(os.path.join(shp_file_path)):
      if shp_file.endswith(".shp"):
        shp_file_name = shp_file
  shp_lyr_check = gpd.read_file(os.path.join(shp_file_path,shp_file_name))

  print('Shapefile CRS: ', shp_lyr_check.crs)

  if shp_lyr_check.crs !=  'EPSG:4326':
    # reproject
    shp_lyr_crs = shp_lyr_check.to_crs(epsg=4326)
    print('Shapefile layer has been reprojected to match shapefile')
  else:
    shp_lyr_crs = shp_lyr_check
    print('Coordinate systems match!')

  shp_lyr_crs.to_file(os.path.join(shp_file_path,shp_file_name))
  print('\n-----------------------------------------------------------------------------------------------')
  print('(3b) Determine intersecting polygons')
  print('-----------------------------------------------------------------------------------------------')
  g1 = gpd.GeoDataFrame.from_file(os.path.join(product_path,f"finalcat_info_{variable['version_num_shp']}.shp"))
  g2 = gpd.GeoDataFrame.from_file(shp_file_path)
  g2["dissolve"] = 1
  boundary = g2[['dissolve', 'geometry']]
  studyArea = boundary.dissolve(by='dissolve')
  studyArea['geometry'] = studyArea.geometry.buffer(-0.005)

  data = []
  for index, crim in g1.iterrows():
      for index2, popu in studyArea.iterrows():
        if crim['geometry'].intersects(popu['geometry']):
            data.append({'geometry': crim['geometry'].intersection(popu['geometry']), 'SubId':crim['SubId']})

  df = gpd.GeoDataFrame(data,columns=['geometry', 'SubId'])

  sub_ids = df['SubId'].unique().tolist()
  test_df = g1[g1['SubId'].isin(sub_ids)]
  test_df.to_file(os.path.join(shp_file_path))

In [11]:
def format_shapefile(shp_file_path):
  print('\n-----------------------------------------------------------------------------------------------')
  print('Format Shapefile')
  print('-----------------------------------------------------------------------------------------------')

  # find name of shapefile
  for shp_file in os.listdir(os.path.join(shp_file_path)):
      if shp_file.endswith(".shp"):
        shp_file_name = shp_file

  studyArea_bound = gpd.read_file(os.path.join(shp_file_path,shp_file_name))
  studyArea_bound["dissolve"] = 1

  boundary = studyArea_bound[['dissolve', 'geometry']]
  cont_studyArea = boundary.dissolve(by='dissolve')

  # remove contents in folder
  for f in glob (os.path.join(shp_file_path,'*')):
    os.remove(f)

  cont_studyArea.to_file(os.path.join(variable['main_dir'], 'shapefile','studyArea_outline.shp'))

In [12]:
def visualize_shapefile(shp_file_path):
  print('\n-----------------------------------------------------------------------------------------------')
  print('Visualize Shapefile')
  print('-----------------------------------------------------------------------------------------------')

  shp_boundary = gpd.read_file(os.path.join(shp_file_path))

  # check projection
  if shp_boundary.crs !=  'EPSG:4326':
    # reproject
    shp_lyr_crs = shp_boundary.to_crs(epsg=4326)
    print('Shapefile layer has been reprojected to match shapefile')
  else:
    shp_lyr_crs = shp_boundary
    print('Coordinate systems match!')

  # determine the boundary of the provided shapefile
  bounds = shp_lyr_crs.bounds
  west, south, east, north = bounds = bounds.loc[0]
  shp_bounds = [south,west]

  map = folium.Map(location=shp_bounds, zoom_start=10)
  folium.GeoJson(data=shp_boundary["geometry"]).add_to(map)
  display(map)

  print('\n-----------------------------------------------------------------------------------------------')
  print('Shapefile complete!')
  print('-----------------------------------------------------------------------------------------------')

In [13]:
def remove_temp_data(main_dir, temporary_dir,product_path):
  print('\n-----------------------------------------------------------------------------------------------')
  print('Remove unnecessary files')
  print('-----------------------------------------------------------------------------------------------')

  if os.path.exists(temporary_dir):
    shutil.rmtree(temporary_dir)
  if os.path.exists(product_path):
    shutil.rmtree(product_path)
  zip_files_rm = ((glob(os.path.join(product_path+"*.zip"))))
  for files_rm in zip_files_rm:
    os.remove(files_rm)

In [16]:
main_dir = '/content/google_drive/MyDrive/Developers_Magpie_Workflow'

with open(os.path.join(main_dir,"configuration_file.json"), "r") as f:
    variable = json.load(f)

In [17]:
if variable['generate_shapefile'] == "yes":
  if variable['gauge_name'] == "NA":
    # step 1
    lat, lon = studyArea_location(variable['city_name_shp'], variable['define_lat_shp'], variable['define_lon_shp'])
    # step 1a
    interactive_gauge_map(variable['interactive_map_response'], lat, lon)
    # step 1b
    product_path = download_routing_product_lat_lon(variable['product_name_shp'], lat, lon)
  else:
    # step 1
    product_path = download_routing_product_gauge(variable['product_name_shp'], variable['gauge_name_shp'])
  # step 2 & 3
  if variable['product_name'] == "CLRH":
    extract_drainage_area_CLRH(variable['most_down_stream_subbasin_ids_shp'], variable['most_up_stream_subbasin_ids_shp'], product_path)
  else:
    extract_drainage_area(variable['most_down_stream_subbasin_ids_shp'], variable['most_up_stream_subbasin_ids_shp'], product_path)
  # step 4
  shp_file_path = os.path.join(variable['main_dir'], 'shapefile')
  visualize_shapefile(shp_file_path)
  # step 5
  remove_temp_data(variable['main_dir'], variable['temporary_dir'],product_path)
else:
  # generate drive folder
  shp_dir = os.path.join(variable['main_dir'], 'shapefile')
  shp_path = os.path.isdir(shp_dir)
  if not shp_path:
    os.makedirs(shp_dir)
    print("created  folder: ", shp_dir)
  print('\n-----------------------------------------------------------------------------------------------')
  print('Upload Shapefile')
  print('-----------------------------------------------------------------------------------------------')
  print(f'drag-and-drop shapefile into the following folder: {shp_dir}')
  response = input("Have you uploaded the study area shapefile (yes or no): ")
  if response == "yes" and variable['format_for_BasinMaker'] == "yes":
    shp_file_path = os.path.join(variable['main_dir'], 'shapefile')
    #format_shapefile(shp_file_path)
    # step 1
    lat, lon = studyArea_location(variable['city_name_shp'], variable['define_lat_shp'], variable['define_lon_shp'])
    # step 2
    product_path = download_routing_product_lat_lon(variable['product_name_shp'], lat, lon)
    # step 3
    intersecting_polygons(shp_file_path,product_path)
    # step 4
    format_shapefile(shp_file_path)
    # remove temporary fi
    remove_temp_data(variable['main_dir'], variable['temporary_dir'],product_path)
    # step 5
    visualize_shapefile(shp_file_path)
  elif response == "yes" and variable['format_for_BasinMaker'] == "no":
    shp_file_path = os.path.join(variable['main_dir'], 'shapefile')
    # step 4
    format_shapefile(shp_file_path)
    # step 5
    visualize_shapefile(shp_file_path)


-----------------------------------------------------------------------------------------------
Upload Shapefile
-----------------------------------------------------------------------------------------------
drag-and-drop shapefile into the following folder: /content/google_drive/MyDrive/Developers_Magpie_Workflow/shapefile
Have you uploaded the study area shapefile (yes or no): yes

-----------------------------------------------------------------------------------------------
(1) Collect study area location information
-----------------------------------------------------------------------------------------------
Study area coordinates: 49.4896314 -113.9471472

-----------------------------------------------------------------------------------------------
(2) Download routing product
-----------------------------------------------------------------------------------------------
product_name:  NALRP
The needed product locates at: /content/drainage_region_0004_v2-1
The Subbasin Id of

<ipython-input-10-0fa7def1beb8>:30: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  studyArea['geometry'] = studyArea.geometry.buffer(-0.005)



-----------------------------------------------------------------------------------------------
Format Shapefile
-----------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------
Visualize Shapefile
-----------------------------------------------------------------------------------------------
Coordinate systems match!



-----------------------------------------------------------------------------------------------
Shapefile complete!
-----------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------
Remove unnecessary files
-----------------------------------------------------------------------------------------------
